In [1]:
from agilerl.utils.utils import create_population
from agilerl.vector.pz_async_vec_env import AsyncPettingZooVecEnv
import torch
import supersuit as ss

from mcrafter import Env
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the network configuration
NET_CONFIG = {
        "arch": "cnn",  # Network architecture
        "hidden_size": [128, 64],  # Slightly larger final dense layer
        "channel_size": [32, 64, 64],  # Uniform channel size
        "kernel_size": [8, 4, 3],      # Same kernel sizes
        "stride_size": [4, 2, 1],      # Same strides
        "normalize": True,  # Normalize image from range [0,255] to [0,1]
    }

# Define the initial hyperparameters
INIT_HP = {
    # Swap image channels dimension from last to first [H, W, C] -> [C, H, W]
    "CHANNELS_LAST": True,
    "BATCH_SIZE": 64,  # Batch size
    "O_U_NOISE": True,  # Ornstein Uhlenbeck action noise
    "EXPL_NOISE": 0.15,  # Action noise scale
    "MEAN_NOISE": 0.0,  # Mean action noise
    "THETA": 0.15,  # Rate of mean reversion in OU noise
    "DT": 0.01,  # Timestep for OU noise
    "LR_ACTOR": 0.001,  # Actor learning rate
    "LR_CRITIC": 0.001,  # Critic learning rate
    "GAMMA": 0.995,  # Discount factor
    "MEMORY_SIZE": 200000,  # Max memory buffer size
    "LEARN_STEP": 32,  # Learning frequency
    "TAU": 0.01,  # For soft update of target parameters
    "POLICY_FREQ": 2,  # Policy frequnecy
    "POP_SIZE": 2,  # Population size
}

num_envs = 8
# Define the simple speaker listener environment as a parallel environment
env = Env(n_players=2, render_mode='human')
env = ss.color_reduction_v0(env, 'full')
env = ss.black_death_v3(env)
env = ss.frame_stack_v1(env, 4)
env = AsyncPettingZooVecEnv([lambda: env for _ in range(num_envs)])
env.reset()

# Configure the multi-agent algo input arguments
try:
    state_dim = [env.single_observation_space(agent).n for agent in env.agents]
    one_hot = True
except Exception:
    state_dim = [env.single_observation_space(agent).shape for agent in env.agents]
    one_hot = False
try:
    action_dim = [env.single_action_space(agent).n for agent in env.agents]
    INIT_HP["DISCRETE_ACTIONS"] = True
    INIT_HP["MAX_ACTION"] = None
    INIT_HP["MIN_ACTION"] = None
except Exception:
    action_dim = [env.single_action_space(agent).shape[0] for agent in env.agents]
    INIT_HP["DISCRETE_ACTIONS"] = False
    INIT_HP["MAX_ACTION"] = [env.single_action_space(agent).high for agent in env.agents]
    INIT_HP["MIN_ACTION"] = [env.single_action_space(agent).low for agent in env.agents]

# Not applicable to MPE environments, used when images are used for observations (Atari environments)
if INIT_HP["CHANNELS_LAST"]:
    state_dim = [
        (state_dim[2], state_dim[0], state_dim[1]) for state_dim in state_dim
    ]

# Append number of agents and agent IDs to the initial hyperparameter dictionary
INIT_HP["N_AGENTS"] = env.num_agents
INIT_HP["AGENT_IDS"] = env.agents

# Create a population ready for evolutionary hyper-parameter optimisation
pop = create_population(
    "MADDPG",
    state_dim,
    action_dim,
    one_hot,
    NET_CONFIG,
    INIT_HP,
    population_size=INIT_HP["POP_SIZE"],
    num_envs=num_envs,
    device=device,
)

In [2]:
from agilerl.components.multi_agent_replay_buffer import MultiAgentReplayBuffer

field_names = ["state", "action", "reward", "next_state", "done"]
memory = MultiAgentReplayBuffer(
    INIT_HP["MEMORY_SIZE"],
    field_names=field_names,
    agent_ids=INIT_HP["AGENT_IDS"],
    device=device,
)

In [3]:
from agilerl.hpo.tournament import TournamentSelection

tournament = TournamentSelection(
    tournament_size=2,  # Tournament selection size
    elitism=True,  # Elitism in tournament selection
    population_size=INIT_HP["POP_SIZE"],  # Population size
    eval_loop=1,  # Evaluate using last N fitness scores
)

In [4]:
from agilerl.hpo.mutation import Mutations

mutations = Mutations(
    algo="MADDPG",
    no_mutation=0.2,  # Probability of no mutation
    architecture=0.2,  # Probability of architecture mutation
    new_layer_prob=0.2,  # Probability of new layer mutation
    parameters=0.2,  # Probability of parameter mutation
    activation=0,  # Probability of activation function mutation
    rl_hp=0.2,  # Probability of RL hyperparameter mutation
    rl_hp_selection=[
        "lr",
        "learn_step",
        "batch_size",
    ],  # RL hyperparams selected for mutation
    mutation_sd=0.1,  # Mutation strength
    agent_ids=INIT_HP["AGENT_IDS"],
    arch=NET_CONFIG["arch"],
    rand_seed=1,
    device=device,
)

In [5]:
from agilerl.training.train_multi_agent import train_multi_agent
import gymnasium as gym
import torch

trained_pop, pop_fitnesses = train_multi_agent(
    env=env,  # Pettingzoo-style environment
    env_name='mcrafter-2agents',  # Environment name
    algo="MADDPG",  # Algorithm
    pop=pop,  # Population of agents
    memory=memory,  # Replay buffer
    INIT_HP=INIT_HP,  # IINIT_HP dictionary
    net_config=NET_CONFIG,  # Network configuration
    swap_channels=INIT_HP['CHANNELS_LAST'],  # Swap image channel from last to first
    max_steps=350_000,  # Max number of training steps
    evo_steps=5000,  # Evolution frequency
    eval_steps=None,  # Number of steps in evaluation episode
    eval_loop=1,  # Number of evaluation episodes
    learning_delay=1000,  # Steps before starting learning
    target=4000.,  # Target score for early stopping
    tournament=tournament,  # Tournament selection object
    mutation=mutations,  # Mutations object
    wb=False,  # Weights and Biases tracking
)


Training...


  1%|1         | 5000/350000 [  00:51<  56:26, 101.89step/s]


DateTime, now, H:m:s-u 22 : 10 : 11 - 10192
Steps 148.1708393912262 per sec, 8890.250363473571 per min.

                --- Global Steps 10000 ---
                Fitness:	['1356.00', '1356.00']
                Score:		[1113.875, 1317.375]
                5 fitness avgs:	['1356.00', '1356.00']
                10 score avgs:	['1317.38', '1317.38']
                Agents:		[1, 2]
                Steps:		[5000, 5000]
                Mutations:	['arch', 'param']


  3%|2         | 10000/350000 [  01:40<  53:01, 106.86step/s]


DateTime, now, H:m:s-u 22 : 10 : 57 - 297448
Steps 175.78257879544648 per sec, 10546.95472772679 per min.

                --- Global Steps 20000 ---
                Fitness:	['2034.00', '2034.00']
                Score:		[1356.0, 1410.5]
                5 fitness avgs:	['1695.00', '1695.00']
                10 score avgs:	['1395.00', '1395.00']
                Agents:		[2, 3]
                Steps:		[10000, 10000]
                Mutations:	['lr_actor', 'None']


...


DateTime, now, H:m:s-u 23 : 9 : 48 - 488984
Steps 186.55854196550246 per sec, 11193.512517930149 per min.

                --- Global Steps 680000 ---
                Fitness:	['2034.00', '2034.00']
                Score:		[2029.5, 1958.5]
                5 fitness avgs:	['2034.00', '2034.00']
                10 score avgs:	['1970.40', '1970.40']
                Agents:		[68, 69]
                Steps:		[340000, 340000]
                Mutations:	['param', 'None']


 99%|#########8| 345000/350000 [1:01:40<  01:04, 77.70step/s]


DateTime, now, H:m:s-u 23 : 10 : 43 - 956180
Steps 186.46453192668133 per sec, 11187.87191560088 per min.

                --- Global Steps 690000 ---
                Fitness:	['2034.00', '2034.00']
                Score:		[1020.125, 2033.75]
                5 fitness avgs:	['2034.00', '2034.00']
                10 score avgs:	['2033.80', '1222.90']
                Agents:		[69, 70]
                Steps:		[345000, 345000]
                Mutations:	['None', 'arch']


100%|##########| 350000/350000 [1:02:33<  00:00, 93.23step/s]


DateTime, now, H:m:s-u 23 : 11 : 37 - 486969
Steps 186.46943508243103 per sec, 11188.166104945863 per min.

                --- Global Steps 700000 ---
                Fitness:	['2034.00', '2034.00']
                Score:		[2017.25, 2034.0]
                5 fitness avgs:	['2034.00', '2034.00']
                10 score avgs:	['1830.60', '1830.60']
                Agents:		[70, 71]
                Steps:		[350000, 350000]
                Mutations:	['None', 'bs']
